In [1]:
!pip install -qq mediapipe --user

In [5]:
import cv2
import mediapipe as mp

In [11]:
# 얼굴 감지 객체 설정 // 그리기 객체 초기화 // 그리기도 설정
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

## 웹캠 객체 초기화 // 비디오 시작
cap = cv2.VideoCapture(0)

# 얼굴 감지 모델 설정
with mp_face_detection.FaceDetection(model_selection=0,
                                     min_detection_confidence = 0.5
                                     ) as face_detection:
    while cap.isOpened():
        success, image = cap.read()

        if not success:
            print("웹캠 이미지 로드 실패")
            continue # 아래 코드 무시

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        results = face_detection.process(image)

        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        print(results)

        if results.detections:
            for detection in results.detections:
                mp_drawing.draw_detection(image, detection)
        
        cv2.imshow("face detection", image)

        if cv2.waitKey(5) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [2]:
# MediaPipe 얼굴 메시(face mesh) 솔루션과 그리기 유틸리티를 초기화
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# 각 눈의 랜드마크 인덱스를 정의
LEFT_EYE_INDICES = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
RIGHT_EYE_INDICES = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246]

# 웹캠 시작
cap = cv2.VideoCapture(0)

# FaceMesh 모델을 초기화. 
with mp_face_mesh.FaceMesh(
    max_num_faces=1, # 최대 얼굴 수
    refine_landmarks=True, # 각 부위별 특징을 세밀하게 추출할지 결정
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5) as face_mesh:
    
    # 비디오가 열려 있을 경우 프레임 계속 읽어오기
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            continue  # 프레임 읽기 실패 시 다음 프레임으로

        # MediaPipe는 RGB이미지 사용 - BGR에서 RGB로색상 변환
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Face Mesh 처리 결과
        results = face_mesh.process(image)

        # 그리기 작업을 위해 이미지를 다시 BGR로 변환
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # 필요한 부위에 그려줘야 - 랜드마크 선택해서 그려주면
        if results.multi_face_landmarks:

            # 검출된 각 얼굴에 랜드마크 표시
            for face_landmarks in results.multi_face_landmarks:

                # 왼쪽 눈을 파란색으로 그리기
                for idx in LEFT_EYE_INDICES:
                    point = face_landmarks.landmark[idx]
                    x = int(point.x * image.shape[1])
                    y = int(point.y * image.shape[0])
                    cv2.circle(image, (x, y), 1, (255, 0, 0), -1)

                # 오른쪽 눈을 녹색으로 그리기
                for idx in RIGHT_EYE_INDICES:
                    point = face_landmarks.landmark[idx]
                    x = int(point.x * image.shape[1])
                    y = int(point.y * image.shape[0])
                    cv2.circle(image, (x, y), 1, (0, 255, 0), -1)

        # 최종 결과 표시
        cv2.imshow('Eyes Landmarks', image)

        # 'q' 키를 누르면 반복을 종료합니다.
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

# 자원을 해제합니다.
cap.release()
cv2.destroyAllWindows()

In [8]:
import numpy as np
# MediaPipe Face Mesh 초기화
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5)

glasses_img = cv2.imread('glasses.png', cv2.IMREAD_UNCHANGED)

# 웹캠 캡처 객체 생성
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break  # 프레임을 제대로 읽지 못하면 루프 탈출

    # BGR 이미지를 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 얼굴 랜드마크 검출
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            h, w, _ = frame.shape
            # 랜드마크 인덱스: 왼쪽 눈 362, 오른쪽 눈 133
            left_eye = face_landmarks.landmark[267]
            right_eye = face_landmarks.landmark[317]

            # 눈의 중심 위치 계산
            eye_center_x = int((left_eye.x + right_eye.x) / 2 * w)
            eye_center_y = int((left_eye.y + right_eye.y) / 2 * h)

            # 두 눈 사이의 x, y 좌표 차이 계산
            delta_x = left_eye.x - right_eye.x
            delta_y = left_eye.y - right_eye.y

            # 안경 이미지의 크기 조정 (여기서는 두 눈 사이의 거리를 기준으로 조정합니다)
            eye_distance = int(np.linalg.norm(np.array([delta_x, delta_y]) * [w, h]))
            glasses_width = int(eye_distance * 5) # 안경 크기가 너무 작게 표시된다면 이 값을 수정
            scale_factor = glasses_width / glasses_img.shape[1]
            resized_glasses = cv2.resize(glasses_img, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_AREA)

            # 안경 이미지의 알파 채널과 RGB 채널 분리
            glasses_alpha = resized_glasses[:, :, 3] / 255.0
            glasses_color = resized_glasses[:, :, :3]

            # 안경 위치 조정
            x = eye_center_x - glasses_width // 2
            y = eye_center_y - resized_glasses.shape[0] // 2

            # 알파 블렌딩을 통한 안경 이미지 합성
            for c in range(0, 3):
                frame[y:y+resized_glasses.shape[0], x:x+glasses_width, c] = glasses_alpha * glasses_color[:, :, c] + (1-glasses_alpha) * frame[y:y+resized_glasses.shape[0], x:x+glasses_width, c]
            
            # 아래와 같이 해도 되지만.. 정수형은 정확하지 않습니다.
            # glasses_alpha = resized_glasses[:, :, 3]
            # mask_inv = cv2.bitwise_not(glasses_alpha)
            # resized_glasses_rgb = resized_glasses[:, :, :3]

            # # 배경 영역을 준비합니다.
            # background = cv2.bitwise_and(frame[y:y+resized_glasses.shape[0], x:x+glasses_width], frame[y:y+resized_glasses.shape[0], x:x+glasses_width], mask=mask_inv)

            # # 안경 영역을 준비합니다.
            # glasses_area = cv2.bitwise_and(resized_glasses_rgb, resized_glasses_rgb, mask=glasses_alpha)

            # # 안경 영역과 배경 영역을 합성합니다.
            # combined = cv2.add(background, glasses_area)

            # # 합성된 이미지를 원본 이미지에 적용합니다.
            # frame[y:y+resized_glasses.shape[0], x:x+glasses_width] = combined

    # 결과 표시
    cv2.imshow('Glasses Overlay', frame)

    # 'q'를 누르면 루프 탈출
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [12]:
print(mp_face_mesh.FACEMESH_LIPS)

frozenset({(270, 409), (317, 402), (81, 82), (91, 181), (37, 0), (84, 17), (269, 270), (321, 375), (318, 324), (312, 311), (415, 308), (17, 314), (61, 146), (78, 95), (0, 267), (82, 13), (314, 405), (178, 87), (267, 269), (61, 185), (14, 317), (88, 178), (185, 40), (405, 321), (13, 312), (324, 308), (409, 291), (146, 91), (87, 14), (78, 191), (95, 88), (311, 310), (39, 37), (40, 39), (402, 318), (191, 80), (80, 81), (310, 415), (181, 84), (375, 291)})
